In [ ]:
#!pip install requests
#!pip install beautifulsoup4
#!pip install fake_useragent
#!pip install html2text
#!pip install pymorphy2 
#!pip install razdel
#!pip install pymystem3
#!pip install xgboost
#!pip install transformers
#!pip install torch
#!pip install tqdm
#!pip3 install torch torchvision torchaudio
#!pip install simplemma
#!pip install tensorflow

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import time
from time import sleep
import random
import re
import os
from datetime import datetime, timedelta, timezone
from pymystem3 import Mystem
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
from tqdm import tqdm
import sqlite3
from simplemma import lemmatize
from ukr_stemmer import UkrainianStemmer
from tree_stem import stem_word
import sqlite3

## Створюємо об'єкт із посиланнями на обрані ЗМІ

In [ ]:
links = {"korrespondent":"https://ua.korrespondent.net/all/",
"ukrpravda":"https://www.pravda.com.ua/news/",
"censor":"https://censor.net/ua/news/all"}
pages = {}

In [ ]:
for key, link in links.items():
    try:
        # Використання fake_useragent для генерації фальшивого агента користувача
        headers = {'User-Agent': UserAgent().random}
        
        response = requests.get(link, headers=headers)
        response.raise_for_status()  # Перевірка на наявність помилок у відповіді сервера

        # Збереження вмісту сторінки в словник
        pages[key] = response.text

        print(f"Запит для {key} успішно виконано.")
    except requests.exceptions.RequestException as e:
        print(f"Помилка при виконанні запиту для {key}: {e}")

# Тепер змінна 'pages' містить вміст сторінок для кожного з посилань


## Отримуємо перелік посилань від видання korrespondent

In [ ]:
# Отримуємо HTML-код зі сторінки 'korrespondent'
korrespondent_html = pages['korrespondent']

# Використовуємо BeautifulSoup для парсингу HTML
soup = BeautifulSoup(korrespondent_html, 'html.parser')

# Знаходимо блок <div class="articles-list">
articles_list = soup.find('div', class_='articles-list')

# Ініціалізуємо пустий список для збереження результатів
result_list = []

# Ітеруємося по всіх статтях у блоку
for link_elem in articles_list.select('div.article__title h3 a'):
    # Отримуємо посилання та назву статті
    link = link_elem['href']
    title = link_elem.get_text(strip=True)

    # Шукаємо батьківський елемент <div class="article"> для визначення часу
    article_elem = link_elem.find_parent('div', class_='article')
    if article_elem:
        time_elem = article_elem.find('div', class_='article__date')
        if time_elem:
            time = time_elem.get_text(strip=True)
            match = re.search(r'\b\d{2}:\d{2}\b', time)
            if match:
                time = match.group(0)
        else:
            time = "N/A"
    else:
        time = "N/A"

    # Додаємо дані до результату
    result_list.append({'link': link, 'title': title, 'time': time})

print(result_list)

In [ ]:
# Створюємо DataFrame на основі отриманих даних
korrespondent_df = pd.DataFrame(result_list)

korrespondent_df = korrespondent_df.dropna()

# Виводимо отриманий DataFrame
korrespondent_df

## Отримуємо перелік посилань від видання Ukrpravda

In [ ]:
# Використовуємо BeautifulSoup для парсингу HTML-коду
soup = BeautifulSoup(pages['ukrpravda'], 'html.parser')

# Знаходимо всі елементи <div> із class="article_news_list"
article_divs = soup.find_all('div', class_='article_news_list')

# Ініціалізуємо порожній список для зберігання результатів
parsed_pages = []

# Ітеруємося по кожному елементу із class="article_news_list"
for article_div in article_divs:
    # Отримуємо значення часу
    time_element = article_div.find('div', class_='article_time')
    time = time_element.text.strip() if time_element else None

    # Пропускаємо елемент, якщо відсутній time_element
    if time is None:
        continue

    # Отримуємо посилання
    link_element = article_div.find('div', class_='article_header').find('a')
    link = link_element['href'] if link_element else None

    # Перевіряємо, чи посилання є відносним шляхом
    if link and not link.startswith('https'):
        # Формуємо повний URL
        full_link = f'https://www.pravda.com.ua{link}' if link else None
    else:
        full_link = link

    # Додаємо інформацію до списку
    parsed_pages.append({"time": time, "link": full_link})

# Результат зберігається у змінній parsed_pages
#print(parsed_pages)


In [ ]:
# Використовуємо BeautifulSoup для парсингу HTML-коду
soup = BeautifulSoup(pages['ukrpravda'], 'html.parser')

# Знаходимо всі елементи <div> із class="article_news_list"
article_divs = soup.find_all('div', class_='article_news_list')

# Ініціалізуємо порожній список для зберігання результатів
parsed_pages = []

# Регулярний вираз для видобування дати з посилань
date_regex = re.compile(r'/(\d{4}/\d{1,2}/\d{1,2})/')

# Ітеруємося по кожному елементу із class="article_news_list"
for article_div in article_divs:
    # Отримуємо значення часу
    time_element = article_div.find('div', class_='article_time')
    time = time_element.text.strip() if time_element else None

    # Пропускаємо елемент, якщо відсутній time_element
    if time is None:
        continue

    # Отримуємо посилання
    link_element = article_div.find('div', class_='article_header').find('a')
    link = link_element['href'] if link_element else None

    # Перевіряємо, чи посилання є відносним шляхом
    if link and not link.startswith(('https://', 'http://', 'www.')):
        # Формуємо повний URL
        full_link = f'https://www.pravda.com.ua{link}' if link else None
        # Отримуємо заголовок, уникаючи тексту вкладених у посилання елементів <em>
        title_element = link_element.find_all(string=True, recursive=False)
        title = ' '.join(title_element).strip() if title_element else None

        # Додаємо інформацію до списку
        parsed_pages.append({"time": time, "link": full_link, "title": title})

# Результат зберігається у змінній parsed_pages
print(parsed_pages)


In [ ]:
# Створюємо DataFrame з нових даних
ukrpravda_df = pd.DataFrame(parsed_pages)
ukrpravda_df = ukrpravda_df.dropna()
ukrpravda_df

## Отримуємо перелік посилань від видання Censor

In [ ]:
# Отримуємо HTML-код зі сторінки 'censor'
censor_html = pages['censor']

In [ ]:
# Ініціалізація об'єкта BeautifulSoup
soup = BeautifulSoup(censor_html, 'html.parser')

# Створення порожнього списку для об'єктів
result = []

# Знаходження всіх елементів з класом "news-list-item"
news_items = soup.find_all('div', class_='news-list-item')

# Ітерація по знайденим елементам
for item in news_items:
    # Отримання необхідних даних
    link = item.find('a', class_='news-list-item__link')['href']
    title = item.find('span', class_='news-list-item__title').text.strip()
    time_str = item.find('span', class_='main-items-text__time').find('time', class_='g-time')['datetime']

    # Декодування часу з рядка у об'єкт datetime
    time = datetime.fromisoformat(time_str).strftime('%H:%M')

    # Створення об'єкта і додавання його до списку
    result.append({'link': link, 'title': title, 'time': time})

# Виведення результату
print(result)

In [ ]:
# Створюємо DataFrame з нових даних
censor_df = pd.DataFrame(result)
censor_df = censor_df.dropna()
censor_df

## Функція для виконання запитів за посиланнями з датафрейму

In [ ]:
def scrape_url(url):
    try:
        headers = {'User-Agent': UserAgent().random}
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        print(f"Запит для {url} успішно виконано.")
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Помилка при виконанні запиту для {url}: {e}")
        return None

def process_dataframe(df):
    df['response'] = None

    for i, url in enumerate(df['link']):
        df.loc[i, 'response'] = scrape_url(url)

        sleep_time = random.uniform(2, 15)
        sleep(sleep_time)

    return df

## Отримуємо дані з Укрправди

In [ ]:
process_dataframe(ukrpravda_df)

In [ ]:
from bs4 import BeautifulSoup
import re
from datetime import datetime

# Використовуємо BeautifulSoup для парсингу HTML-коду
soup = BeautifulSoup(pages['ukrpravda'], 'html.parser')

# Знаходимо всі елементи <div> із class="article_news_list"
article_divs = soup.find_all('div', class_='article_news_list')

# Ініціалізуємо порожній список для зберігання результатів
parsed_pages = []

# Ітеруємося по кожному елементу із class="article_news_list"
for article_div in article_divs:
    # Отримуємо значення часу
    time_element = article_div.find('div', class_='article_time')
    time = time_element.text.strip() if time_element else None

    # Пропускаємо елемент, якщо відсутній time_element
    if time is None:
        continue

    # Отримуємо посилання
    link_element = article_div.find('div', class_='article_header').find('a')
    link = link_element['href'] if link_element else None

    # Перевіряємо, чи посилання є відносним шляхом
    if link and not link.startswith(('https://', 'http://', 'www.')):
        # Формуємо повний URL
        full_link = f'https://www.pravda.com.ua{link}' if link else None
        # Отримуємо заголовок, уникайте тексту вкладених у посилання елементів <em>
        title_element = link_element.find_all(string=True, recursive=False)
        title = ' '.join(title_element).strip() if title_element else None

        # Додаємо інформацію до списку
        parsed_pages.append({"time": time, "link": full_link, "title": title})

# Результат зберігається у змінній parsed_pages
print(parsed_pages)


In [ ]:
ukrpravda_df['views'] = pd.Series(dtype=float)
ukrpravda_df['text'] = pd.Series(dtype=str)

# Функція для витягування views та text з HTML-коду
def extract_views_text(html):
    if isinstance(html, str):
        soup = BeautifulSoup(html, 'html.parser')
    
        # Отримання значення з елемента, що має клас "post_views"
        views_element = soup.find('div', class_='post_views')
        views = views_element.text.strip() if views_element else None
    
        # Отримання тексту з елемента, що має клас "post_text"
        text_element = soup.find('div', class_='post_text')
        text = ' '.join([str(element) for element in text_element.contents]) if text_element else None
    
        return views, text
    else:
        return None, None

# Задаємо умову для обробки рядків, де 'response' не є NaN
rows_to_process = ukrpravda_df['response'].notna()

# Створюємо новий DataFrame з копією даних
updated_df = ukrpravda_df.copy()

# Функція для оновлення значень 'views' та 'text' в рядках
def update_views_text(row):
    if isinstance(row['response'], str):
        views, text = extract_views_text(row['response'])
        row['views'] = views
        row['text'] = text
    return row

# Застосовуємо функцію до нового DataFrame
updated_df.loc[rows_to_process] = updated_df.loc[rows_to_process].apply(update_views_text, axis=1)

# Виводимо оновлений DataFrame
ukrpravda_df = updated_df.copy()


In [ ]:
import pandas as pd
import re

# Припустимо, що у вас вже є Pandas DataFrame з ім'ям ukrpravda_df

# Функція для очищення тексту від тегів та спеціальних символів


def clean_text(text):
    if isinstance(text, str):  # Перевірка, чи поле є рядком
        # Видаляємо HTML теги
        text = re.sub(r'<.*?>', '', text)
        # Видаляємо спеціальні символи \n, \t, \r
        text = re.sub(r'[\n\t\r]', ' ', text)
        # Заміна \' на '
        text = text.replace(r"\'", "'")
        # Видалення символів \xa0
        text = text.replace('\xa0', ' ')
        # Видалення слів
        text = text.replace('ВІДЕО ДНЯ', '')
    return text


# Застосовуємо функцію clean_text до кожної колонки типу текст
for column in ukrpravda_df.select_dtypes(include='object').columns:
    ukrpravda_df[column] = ukrpravda_df[column].apply(clean_text)

# Виводимо оновлений DataFrame
ukrpravda_df


## Отримуємо дані з Кореспондента

In [ ]:
process_dataframe(korrespondent_df)

In [ ]:
# Функція для отримання значення з <div class="post-item__views"><span>
def get_views(response):
    soup = BeautifulSoup(response, 'html.parser')
    views_element = soup.find('div', class_='post-item__views')
    if views_element:
        return views_element.find('span').get_text(strip=True)
    return None

# Функція для отримання об'єднаного тексту із усіх <p> всередині <div class="post-item__text">
def get_text(response):
    soup = BeautifulSoup(response, 'html.parser')
    text_element = soup.find('div', class_='post-item__text')
    if text_element:
        paragraphs = text_element.find_all('p')
        # Додаємо всі теги <p>, окрім останнього, із пробілами перед текстом
        return ' '.join(f' {p.get_text(separator=" ", strip=True)}' for p in paragraphs[:-1])
    return None

# Додаємо нові колонки 'views' та 'text'
korrespondent_df['views'] = korrespondent_df['response'].apply(get_views)
korrespondent_df['text'] = korrespondent_df['response'].apply(get_text)


# Перевірка на пусті рядки у колонці "text" та видалення їх
korrespondent_df = korrespondent_df.dropna(subset=['text'])

# Перевірка на недруковані символи у колонці "text" та видалення їх
korrespondent_df = korrespondent_df[korrespondent_df['text'].astype(str).str.strip() != '']


# Виводимо оновлений DataFrame
korrespondent_df[['views', 'text']]

In [ ]:
korrespondent_df

## Отримуємо дані з Censor.net

In [ ]:
process_dataframe(censor_df)

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

def extract_data_from_html(row):
    # Отримання HTML-коду з колонки 'response'
    html_code = row['response']

    # Ініціалізація об'єкта BeautifulSoup
    soup = BeautifulSoup(html_code, 'html.parser')

    # Отримання кількості переглядів (views)
    views_span = soup.find('span', class_='main-items-text__count')
    views = int(''.join(filter(str.isdigit, views_span.text))) if views_span else None

    # Отримання тексту з елемента з класом "news-text"
    text_div = soup.find('div', class_='news-text')
    text_paragraphs = text_div.find_all('p') if text_div else []

    # Збираємо текст з усіх текстових входжень, крім останніх двох абзаців
    text = ' '.join(paragraph.text.strip() for paragraph in text_paragraphs[:-2])

    # Повертаємо дані про перегляди та текст
    return pd.Series({'views': views, 'text': text})


# Створення колонок 'views' та 'text'
censor_df[['views', 'text']] = censor_df.apply(extract_data_from_html, axis=1)

# Виведення результату
print(censor_df[['views', 'text']])

## Створення метрик успішності для кожного джерела

In [ ]:
def calculate_popularity_score(df, views_column='views', new_column='popularity_score'):
    # Перетворення значень у колонці 'views' у числовий формат
    df[views_column] = pd.to_numeric(df[views_column], errors='coerce')

    # Функція для розрахунку популярності за кількістю переглядів у відсотках
    def calculate_percentage(views, max_views):
        if max_views == 0:
            return 0  # Якщо немає переглядів, повертаємо 0
        else:
            percentage = (views / max_views) * 100
            return round(percentage)

    # Перевірка наявності даних та видалення рядків з NaN значеннями у колонці з переглядами
    df = df.dropna(subset=[views_column])

    # Знаходження максимального значення переглядів у датафреймі
    max_views = df[views_column].max()

    # Додавання нової колонки 'popularity_score' до датафрейму
    df[new_column] = df.apply(lambda row: calculate_percentage(row[views_column], max_views), axis=1)

    return df

### Укрправда

In [ ]:
ukrpravda_df = calculate_popularity_score(ukrpravda_df, views_column='views', new_column='popularity_score')

### Кореспондент

In [ ]:
korrespondent_df = calculate_popularity_score(korrespondent_df, views_column='views', new_column='popularity_score')

### Censor

In [ ]:
censor_df = calculate_popularity_score(censor_df, views_column='views', new_column='popularity_score')

## Об'єднання датафреймів

In [ ]:
combined_df = pd.concat([ukrpravda_df, korrespondent_df, censor_df], ignore_index=True)
combined_df = combined_df.drop('response', axis=1)
combined_df = combined_df.dropna()
combined_df

## Очистка тексту

In [ ]:
def clean_text(text):
    # Видалення HTML-тегів (якщо вони є)
    text = re.sub(r'<.*?>', '', text)
    
    # Видалення URL-адрес
    text = re.sub(r'http\S+', '', text)
    
    # Видалення числових значень
    text = re.sub(r'\d+', '', text)
    
    # Видалення зайвих пробілів
    text = ' '.join(text.split())
    
    # Видалення пунктуації та інших непотрібних символів
    text = re.sub(r'[^A-Za-zА-Яа-яҐґІіЇїЄєЁёЎўЇїʼ\s]', '', text)
    
    return text

combined_df['cleaned_text'] = combined_df['text'].apply(clean_text)

## Переведення тексту в нижній реєстр

In [ ]:
def lowercase_text(row):
    return row['cleaned_text'].lower()

# Застосуйте функцію до кожного рядка датафрейму
combined_df['lowercase_text'] = combined_df.apply(lowercase_text, axis=1)

## Прибирання стоп-слів

In [ ]:
custom_stop_words = ['а', 'аби', 'абиде', 'абиким', 'абикого', 'абиколи', 'абикому', 'абикуди', 'абихто', 'абичий', 'абичийого', 'абичийому', 'абичим', 'абичию', 'абичия', 'абичиє', 'абичиєму', 'абичиєю', 'абичиєї', 'абичиї', 'абичиїй', 'абичиїм', 'абичиїми', 'абичиїх', 'абичого', 'абичому', 'абищо', 'абияка', 'абияке', 'абиякий', 'абияким', 'абиякими', 'абияких', 'абиякого', 'абиякому', 'абиякою', 'абиякої', 'абияку', 'абиякі', 'абиякій', 'абиякім', 'або', 'абощо', 'авжеж', 'авось', 'ага', 'ад', 'адже', 'аж', 'ажень', 'аз', 'ай', 'але', 'ало', 'амінь', 'ант', 'ану', 'ані', 'аніде', 'аніж', 'анізащо', 'аніким', 'анікого', 'анікогісінько', 'аніколи', 'анікому', 'аніскільки', 'аніхто', 'анічим', 'анічого', 'анічогісінько', 'анічому', 'аніщо', 'аніяка', 'аніяке', 'аніякий', 'аніяким', 'аніякими', 'аніяких', 'аніякого', 'аніякому', 'аніякою', 'аніякої', 'аніяку', 'аніякі', 'аніякій', 'аніякім', 'аніякісенька', 'аніякісеньке', 'аніякісенький', 'аніякісеньким', 'аніякісенькими', 'аніякісеньких', 'аніякісенького', 'аніякісенькому', 'аніякісенькою', 'аніякісенької', 'аніякісеньку', 'аніякісенькі', 'аніякісенькій', 'аніякісенькім', 'аніякісінька', 'аніякісіньке', 'аніякісінький', 'аніякісіньким', 'аніякісінькими', 'аніякісіньких', 'аніякісінького', 'аніякісінькому', 'аніякісінькою', 'аніякісінької', 'аніякісіньку', 'аніякісінькі', 'аніякісінькій', 'аніякісінькім', 'ат', 'ато', 'атож', 'ау', 'ах', 'ач', 'ачей', 'аякже', 'б', 'ба', 'багато', 'багатьма', 'багатьом', 'багатьох', 'баз', 'бай', 'бат', 'бах', 'бац', 'баш', 'бе', 'беж', 'без', 'безперервно', 'бел', 'бер', 'би', 'бир', 'бич', 'близько', 'близько від', 'бо', 'бов', 'бод', 'бодай', 'боз', 'бош', 'був', 'буває', 'буде', 'будем', 'будемо', 'будете', 'будеш', 'буду', 'будуть', 'будь', 'будь ласка', 'будьмо', 'будьте', 'була', 'були', 'було', 'бути', 'бух', 'буц', 'буцім', 'буцімто', 'бі', 'біб', 'більш', 'більше', 'біля', 'в', 'в бік', 'в залежності від', 'в міру', 'в напрямі до', 'в порівнянні з', 'в процесі', 'в результаті', 'в ролі', 'в силу', 'в сторону', 'в супроводі', 'в ході', "в ім'я", 'в інтересах', 'вад', 'важлива', 'важливе', 'важливий', 'важливі', 'вак', 'вам', 'вами', 'ван', 'вас', 'ват', 'ваш', 'ваша', 'ваше', 'вашим', 'вашими', 'ваших', 'вашого', 'вашому', 'вашою', 'вашої', 'вашу', 'ваші', 'вашій', 'вашім', 'ввесь', 'вві', 'вгору', 'вдалині', 'вед', 'верх', 'весь', 'вех', 'вже', 'вздовж', 'ви', 'виз', 'вис', 'височині', 'вище ', 'вйо', 'власне', 'властиво', 'вміти', 'внаслідок', 'вниз', 'внизу', 'во', 'вон', 'вона', 'вони', 'воно', 'восьмий', 'вперед', 'вподовж', 'впоперек', 'впритиск', 'впритул', 'впродовж', 'впрост', 'все', 'всередині', 'всею', 'вслід', 'всупереч', 'всього', 'всьому', 'всю', 'всюди', 'вся', 'всяк', 'всяка', 'всяке', 'всякий', 'всяким', 'всякими', 'всяких', 'всякого', 'всякому', 'всякою', 'всякої', 'всяку', 'всякі', 'всякій', 'всякім', 'всі', 'всій', 'всіляка', 'всіляке', 'всілякий', 'всіляким', 'всілякими', 'всіляких', 'всілякого', 'всілякому', 'всілякою', 'всілякої', 'всіляку', 'всілякі', 'всілякій', 'всілякім', 'всім', 'всіма', 'всіх', 'всією', 'всієї', 'втім', 'ві', 'віг', 'від', 'від імені', 'віддалік від', 'відколи', 'відносно', 'відповідно', 'відповідно до', 'відсотків', 'відтепер', 'відтоді', 'він', 'вісім', 'вісімнадцятий', 'вісімнадцять', 'віт', 'віф', 'віх', 'віц', 'віщо', 'віщось', 'г', 'га', 'гав', 'гаразд', 'ге', 'гез', 'гем', 'геп', 'гет', 'геть', 'гех', 'ги', 'гик', 'гир', 'гич', 'гм', 'го', 'говорив', 'гог', 'гоп', 'гоц', 'гу', 'гуп', 'д', 'да', 'давай', 'давати', 'давно', 'далеко', 'далеко від', 'далі', 'даром', 'два', 'двадцятий', 'двадцять', 'дванадцятий', 'дванадцять', 'двох', 'дві', 'де', "дев'ятий", "дев'ятнадцятий", "дев'ятнадцять", "дев'ять", 'дедалі', 'деким', 'декого', 'деколи', 'декому', 'декотра', 'декотре', 'декотрий', 'декотрим', 'декотрими', 'декотрих', 'декотрого', 'декотрому', 'декотрою', 'декотрої', 'декотру', 'декотрі', 'декотрій', 'декотрім', 'декілька', 'декільком', 'декількома', 'декількох', 'декім', 'десь', 'десятий', 'десять', 'дехто', 'дечий', 'дечийого', 'дечийому', 'дечим', 'дечию', 'дечия', 'дечиє', 'дечиєму', 'дечиєю', 'дечиєї', 'дечиї', 'дечиїй', 'дечиїм', 'дечиїми', 'дечиїх', 'дечого', 'дечому', 'дечім', 'дещо', 'деяка', 'деяке', 'деякий', 'деяким', 'деякими', 'деяких', 'деякого', 'деякому', 'деякою', 'деякої', 'деяку', 'деякі', 'деякій', 'деякім', 'деінде', 'для', 'до', 'добре', 'довго', 'довкола', 'довкіл', 'дог', 'доки', 'допоки', 'допіру', 'досить', 'досі', 'дотепер', 'доти', 'другий', 'друго', 'дуже', 'дякую', 'дійсно', 'діл', 'е', 'еге', 'еж', 'ей', 'ерг', 'ест', 'ет', 'ех', 'еч', 'ж', 'же', 'жоден', 'жодна', 'жодне', 'жодний', 'жодним', 'жодними', 'жодних', 'жодного', 'жодному', 'жодною', 'жодної', 'жодну', 'жодні', 'жодній', 'жоднім', 'жоднісінька', 'жоднісіньке', 'жоднісінький', 'жоднісіньким', 'жоднісінькими', 'жоднісіньких', 'жоднісінького', 'жоднісінькому', 'жоднісінькою', 'жоднісінької', 'жоднісіньку', 'жоднісінькі', 'жоднісінькій', 'жоднісінькім', 'жуз', 'з', 'з метою', 'з нагоди', 'з приводу', 'з розрахунку на', 'з-за', 'з-над', 'з-перед', 'з-поза', 'з-поміж', 'з-понад', 'з-поперед', 'з-посеред', 'з-проміж', 'з-під', 'з-серед', 'за', 'за винятком', 'за допомогою', 'за посередництвом', 'за рахунок', 'завгодно', 'завдяки', 'завжди', 'завше', 'задля', 'зазвичай', 'зайнята', 'зайнятий', 'зайнято', 'зайняті', 'залежно', 'залежно від', 'замість', 'занадто', 'заради', 'зараз', 'зас', 'зате', 'збоку', 'збоку від', 'зважаючи на', 'зверх ', 'зверху', 'звичайно', 'звиш', 'звідки', 'звідкилясь', 'звідкись', 'звідкіль', 'звідкіля', 'звідкілясь', 'звідси', 'звідсіль', 'звідсіля', 'звідти', 'звідтіль', 'звідтіля', 'звідусюди', 'звідусіль', 'звідціля', 'згідно з', 'здається', 'здовж', 'зем', 'зет', 'ззаду', 'зиз', 'зик', 'значить', 'знову', 'зо', 'зовсім', 'зсередини', 'зух', 'зі', 'зіс', 'и', 'ич', 'й', 'ймовірно', 'йно', 'йо', 'його', 'йой', 'йол', 'йому', 'йор', 'йот', 'йох', 'к', 'каже', 'каз', 'кар', 'каф', 'ках', 'ке', 'кед', 'кет', 'кеш', 'кив', 'кий', 'кил', 'ким', 'кимось', 'кимсь', 'ких', 'киш', 'коб', 'коби', 'кого', 'когось', 'кожен', 'кожна', 'кожне', 'кожний', 'кожним', 'кожними', 'кожних', 'кожного', 'кожному', 'кожною', 'кожної', 'кожну', 'кожні', 'кожній', 'кожнім', 'кожнісінька', 'кожнісіньке', 'кожнісінький', 'кожнісіньким', 'кожнісінькими', 'кожнісіньких', 'кожнісінького', 'кожнісінькому', 'кожнісінькою', 'кожнісінької', 'кожнісіньку', 'кожнісінькі', 'кожнісінькій', 'кожнісінькім', 'коли', 'колись', 'коло', 'кому', 'комусь', 'котра', 'котрась', 'котре', 'котресь', 'котрий', 'котрийсь', 'котрим', 'котрими', 'котримись', 'котримось', 'котримсь', 'котрих', 'котрихось', 'котрихсь', 'котрого', 'котрогось', 'котрому', 'котромусь', 'котрою', 'котроюсь', 'котрої', 'котроїсь', 'котру', 'котрусь', 'котрі', 'котрій', 'котрійсь', 'котрім', 'котрімсь', 'котрісь', 'коц', 'коч', 'коштом', 'край', 'краще', 'кру', 'круг', 'кругом', 'крю', 'кря', 'крізь', 'крім', 'куди', 'кудись', 'кудою', 'кілька', 'кільком', 'кількома', 'кількох', 'кім', 'кімось', 'кімсь', 'кінець', 'л', 'лаж', 'лап', 'лас', 'лат', 'ле', 'ледве', 'ледь', 'лет', 'лиш', 'лише', 'лишень', 'лум', 'луп', 'лут', 'льє', 'люди', 'людина', 'ля', 'лі', 'ліворуч від', 'лік', 'лім', 'м', 'мабуть', 'майже', 'мало', 'мати', 'мац', 'ме', 'меж', 'мене', 'менше', 'мені', 'мерсі', 'мет', 'мжа', 'ми', 'мимо ', 'миру', 'мит', 'мною', 'мо', 'мов', 'мовби', 'мовбито', 'могла', 'могли', 'могло', 'мого', 'могти', 'мож', 'може', 'можем', 'можемо', 'можете', 'можеш', 'можна', 'можу', 'можуть', 'можіть', 'мой', 'мол', 'мою', 'моя', 'моє', 'моєму', 'моєю', 'моєї', 'мої', 'моїй', 'моїм', 'моїми', 'моїх', 'му', 'мі', 'міг', 'між', 'мій', 'мільйонів', 'н', 'на', 'на адресу', 'на базі', 'на благо', 'на випадок', 'на відміну від', 'на засадах', 'на знак', 'на зразок', 'на користь', 'на кшталт', 'на межі', 'на основі', 'на противагу', 'на підставі', 'на честь', 'на чолі', 'на ґрунті', 'навколо', 'навкруг', 'навкруги ', 'навкіл', 'навпаки', 'навперейми', 'навпроти', 'навіть', 'навіщо', 'навіщось', 'нагорі', 'над', 'надо', 'надовкола', 'надокола', 'наді', 'назавжди', 'назад', 'назустріч', 'най', 'найбільш', 'нам', 'нами', 'наоколо ', 'наокруг ', 'наокруги ', 'наокіл', 'наперед', 'напередодні', 'напереді', 'наперекір', 'напереріз', 'наприкінці', 'напроти', 'нарешті', 'нарівні з', 'нас', 'насеред', 'насподі', 'наспід', 'настрічу', 'насупроти', 'насупротив ', 'нате', 'наче', 'начеб', 'начебто', 'наш', 'наша', 'наше', 'нашим', 'нашими', 'наших', 'нашого', 'нашому', 'нашою', 'нашої', 'нашу', 'наші', 'нашій', 'нашім', 'не', 'не до', 'не можна', 'неабичим', 'неабичого', 'неабичому', 'неабищо', 'небагато', 'небагатьма', 'небагатьом', 'небагатьох', 'небудь', 'невважаючи', 'невже', 'недалеко', 'недалеко від', 'неж', 'незалежно від', 'незважаючи', 'незважаючи на', 'ней', 'немає', 'немов', 'немовби', 'немовбито', 'неначе', 'неначебто', 'неподалеку', 'неподалеку від', 'неподалечку', 'неподалечку від', 'неподалік', 'неподалік від', 'нерідко', 'нех', 'нехай', 'нещодавно', 'нею', 'неї', 'нижче', 'низько', 'ник', 'ним', 'ними', 'них', 'нич', 'но', 'ну', 'нуг', 'нуд', 'нум', 'нумо', 'нумте', 'ньо', 'нього', 'ньому', 'ню', 'нюх', 'ня', 'няв', 'ні', 'ніби', 'ніби-то', 'нібито', 'ніде', 'ніж', 'нізащо', 'нізвідки', 'нізвідкіля', 'ній', 'ніким', 'нікого', 'нікогісінько', 'ніколи', 'нікому', 'нікотра', 'нікотре', 'нікотрий', 'нікотрим', 'нікотрими', 'нікотрих', 'нікотрого', 'нікотрому', 'нікотрою', 'нікотрої', 'нікотру', 'нікотрі', 'нікотрій', 'нікотрім', 'нікуди', 'нім', 'нінащо', 'ніскільки', 'ніт', 'ніхто', 'нічий', 'нічийна', 'нічийне', 'нічийний', 'нічийним', 'нічийними', 'нічийних', 'нічийного', 'нічийному', 'нічийною', 'нічийної', 'нічийну', 'нічийні', 'нічийній', 'нічийнім', 'нічийого', 'нічийому', 'нічим', 'нічию', 'нічия', 'нічиє', 'нічиєму', 'нічиєю', 'нічиєї', 'нічиї', 'нічиїй', 'нічиїм', 'нічиїми', 'нічиїх', 'нічого', 'нічому', 'ніщо', 'ніяк', 'ніяка', 'ніяке', 'ніякий', 'ніяким', 'ніякими', 'ніяких', 'ніякого', 'ніякому', 'ніякою', 'ніякої', 'ніяку', 'ніякі', 'ніякій', 'ніякім', 'ніякісінька', 'ніякісіньке', 'ніякісінький', 'ніякісіньким', 'ніякісінькими', 'ніякісіньких', 'ніякісінького', 'ніякісінькому', 'ніякісінькою', 'ніякісінької', 'ніякісіньку', 'ніякісінькі', 'ніякісінькій', 'ніякісінькім', 'о', 'об', 'обабіч', 'обаполи', 'обидва', 'обр', 'обік', 'обіруч', 'обіч', 'ов', 'од', 'один', 'одинадцятий', 'одинадцять', 'одна', 'однак', 'одначе', 'одне', 'одним', 'одними', 'одних', 'одно', 'одного', 'одного разу', 'одному', 'одною', 'одної', 'одну', 'одні', 'одній', 'однім', 'однією', 'однієї', 'ож', 'ой', 'окрай', 'окроме', 'округ', 'округи', 'окрім', 'окіл', 'ом', 'он', 'онде', 'онно', 'оно', 'оподаль', 'оподаль від', 'оподалік', 'оподалік від', 'опостін', 'опостінь', 'опроче', 'опріч', 'опріче', 'опісля', 'осе', 'оскільки', 'особливо', 'осторонь', 'ось', 'осісьо', 'от', 'ота', 'отак', 'отака', 'отаке', 'отакий', 'отаким', 'отакими', 'отаких', 'отакого', 'отакому', 'отакою', 'отакої', 'отаку', 'отакі', 'отакій', 'отакім', 'отакісінька', 'отакісіньке', 'отакісінький', 'отакісіньким', 'отакісінькими', 'отакісіньких', 'отакісінького', 'отакісінькому', 'отакісінькою', 'отакісінької', 'отакісіньку', 'отакісінькі', 'отакісінькій', 'отакісінькім', 'отам', 'оте', 'отже', 'отим', 'отими', 'отих', 'ото', 'отого', 'отож', 'отой', 'отому', 'отою', 'отої', 'отсе', 'оттак', 'отто', 'оту', 'отут', 'оті', 'отій', 'отім', 'отією', 'отієї', 'ох', 'оце', 'оцей', 'оцим', 'оцими', 'оцих', 'оцього', 'оцьому', 'оцю', 'оця', 'оці', 'оцій', 'оцім', 'оцією', 'оцієї', 'п', "п'я", "п'ятий", "п'ятнадцятий", "п'ятнадцять", "п'ять", 'па', 'пад', 'пак', 'пек', 'перед', 'передо', 'переді', 'перетака', 'перетаке', 'перетакий', 'перетаким', 'перетакими', 'перетаких', 'перетакого', 'перетакому', 'перетакою', 'перетакої', 'перетаку', 'перетакі', 'перетакій', 'перетакім', 'перший', 'пиж', 'плі', 'по', 'поблизу', 'побік', 'побіля', 'побіч', 'поверх', 'повз', 'повздовж', 'повинно', 'повище', 'повсюди', 'повсюдно', 'подаль від', 'подалі від', 'подекуди', 'подеяка', 'подеяке', 'подеякий', 'подеяким', 'подеякими', 'подеяких', 'подеякого', 'подеякому', 'подеякою', 'подеякої', 'подеяку', 'подеякі', 'подеякій', 'подеякім', 'подовж', 'подібно до', 'поз', 'поза', 'позад', 'позаду', 'позата', 'позате', 'позатим', 'позатими', 'позатих', 'позатого', 'позатой', 'позатому', 'позатою', 'позатої', 'позату', 'позаті', 'позатій', 'позатім', 'позатією', 'позатієї', 'позаяк', 'поздовж', 'поки', 'покрай', 'покіль', 'помежи', 'помимо', 'поміж', 'помість', 'понад', 'понадо', 'понаді', 'понижче', 'пообіч', 'поодаль від', 'поодалік від', 'поперед', 'попереду', 'поперек', 'попліч', 'попри', 'попросту', 'попід', 'пора', 'поруч', 'поряд', 'поряд з', 'порівняно з', 'посеред', 'посередині', 'потрібно', 'потім', 'поуз', 'початку', 'почерез', 'праворуч від', 'пред', 'предо', 'преді', 'прекрасно', 'прецінь', 'при', 'притому', 'причому', 'причім', 'про', 'проз', 'промеж', 'проміж', 'просто', 'проте', 'проти', 'против', 'противно', 'протягом', 'пря', 'пріч', 'пхе', 'пху', 'пі', 'пів', 'півперек', 'під', 'під знаком', 'під приводом', 'під час', 'підо', 'пізніше', 'пім', 'пір', 'після', 'р', 'ради', 'раз', 'разом з', 'разу', 'рано', 'раніш', 'раніш від', 'раніше', 'раніше від', 'раптом', 'ре', 'рет', 'риж', 'рим', 'рип', 'роб', 'року', 'років', 'рос', 'рох', 'році', 'рус', 'рух', 'руч', 'рік', 'с', 'саж', 'саз', 'сак', 'сам', 'сама', 'саме', 'сами', 'самий', 'самим', 'самими', 'самих', 'само', 'самого', 'самому', 'самою', 'самої', 'саму', 'самі', 'самій', 'самім', 'сап', 'сас', 'свого', 'свою', 'своя', 'своє', 'своєму', 'своєю', 'своєї', 'свої', 'своїй', 'своїм', 'своїми', 'своїх', 'свій', 'се', 'себе', 'себто', 'сей', 'сен', 'серед', 'середи', 'середу', 'сеч', 'си', 'сив', 'сиг', 'сиз', 'сик', 'сиріч', 'сих', 'сказав', 'сказала', 'сказати', 'скрізь', 'скільки', 'скільки-то', 'скількись', 'скільком', 'скількома', 'скількомась', 'скількомось', 'скількомсь', 'скількох', 'скількохось', 'скількохсь', 'сли', 'слідом за', 'соб', 'собою', 'собі', 'соп', 'спасибі', 'спереду', 'спочатку', 'справ', 'справді', 'став', 'стосовно', 'стільки', 'стільком', 'стількома', 'стількох', 'су', 'судячи з', 'супроти', 'супротив', 'суть', 'суч', 'суш', 'сьогодні', 'сьомий', 'сюди', 'ся', 'сяг', 'сяк', 'сяка', 'сяке', 'сякий', 'сяким', 'сякими', 'сяких', 'сякого', 'сякому', 'сякою', 'сякої', 'сяку', 'сякі', 'сякій', 'сякім', 'сям', 'сі', 'сім', 'сімнадцятий', 'сімнадцять', 'сіп', 'т', 'та', 'таж', 'так', 'така', 'таке', 'такенна', 'такенне', 'такенний', 'такенним', 'такенними', 'такенних', 'такенного', 'такенному', 'такенною', 'такенної', 'такенну', 'такенні', 'такенній', 'такеннім', 'таки', 'такий', 'таким', 'такими', 'таких', 'такого', 'також', 'такому', 'такою', 'такої', 'таку', 'такі', 'такій', 'такім', 'такісінька', 'такісіньке', 'такісінький', 'такісіньким', 'такісінькими', 'такісіньких', 'такісінького', 'такісінькому', 'такісінькою', 'такісінької', 'такісіньку', 'такісінькі', 'такісінькій', 'такісінькім', 'тал', 'там', 'тамки', 'тамта', 'тамте', 'тамтим', 'тамтими', 'тамтих', 'тамтого', 'тамтой', 'тамтому', 'тамтою', 'тамтої', 'тамту', 'тамті', 'тамтій', 'тамтім', 'тамтією', 'тамтієї', 'тар', 'тат', 'таш', 'тва', 'твого', 'твою', 'твоя', 'твоє', 'твоєму', 'твоєю', 'твоєї', 'твої', 'твоїй', 'твоїм', 'твоїми', 'твоїх', 'твій', 'те', 'тебе', 'тег', 'теж', 'тем', 'тепер', 'теперечки', 'тес', 'теф', 'теє', 'ти', 'тик', 'тил', 'тим', 'тими', 'тисяч', 'тих', 'то', 'тобою', 'тобто', 'тобі', 'того', 'тоді', 'тож', 'той', 'тол', 'тому', 'тому що', 'тот', 'тощо', 'тою', 'тої', 'тра', 'тре', 'треба', 'третій', 'три', 'тринадцятий', 'тринадцять', 'трохи', 'тс', 'тсс', 'ту', 'туди', 'тудою', 'туп', 'тут', 'тутеньки', 'тутечки', 'тутки', 'туф', 'туц', 'тю', 'тюг', 'тюп', 'тяг', 'тяж', 'тям', 'тяп', 'ті', 'тій', 'тільки', 'тім', 'тією', 'у', 'у бік', 'у вигляді', 'у випадку', 'у відповідності до', 'у відповідь на', 'у залежності від', "у зв'язку з", 'у міру', 'у напрямі до', 'у порівнянні з', 'у процесі', 'у результаті', 'у ролі', 'у силу', 'у сторону', 'у супроводі', 'у ході', 'ув', 'увесь', 'уві', 'угу', 'уже', 'узбіч', 'уздовж', 'укр', 'ум', 'унаслідок', 'униз', 'унизу', 'унт', 'уперед', 'уподовж', 'упоперек', 'упритиск до', 'упритул до', 'упродовж', 'упрост', 'ус', 'усе', 'усередині', 'услід', 'услід за', 'усупереч', 'усього', 'усьому', 'усю', 'усюди', 'уся', 'усяк', 'усяка', 'усяке', 'усякий', 'усяким', 'усякими', 'усяких', 'усякого', 'усякому', 'усякою', 'усякої', 'усяку', 'усякі', 'усякій', 'усякім', 'усі', 'усій', 'усіляка', 'усіляке', 'усілякий', 'усіляким', 'усілякими', 'усіляких', 'усілякого', 'усілякому', 'усілякою', 'усілякої', 'усіляку', 'усілякі', 'усілякій', 'усілякім', 'усім', 'усіма', 'усіх', 'усією', 'усієї', 'утім', 'ух', 'ф', "ф'ю", 'фа', 'фаг', 'фай', 'фат', 'фе', 'фед', 'фез', 'фес', 'фет', 'фзн', 'фоб', 'фот', 'фра', 'фру', 'фу', 'фук', 'фур', 'фус', 'фіш', 'х', 'ха', 'хаз', 'хай', 'хап', 'хат', 'хащ', 'хе', 'хет', 'хи', 'хиб', 'хм', 'хо', 'хов', 'хол', 'хон', 'хоп', 'хор', 'хотіти', 'хоч', 'хоча', 'хочеш', 'хро', 'хрю', 'хто', 'хтось', 'ху', 'хуз', 'хук', 'хух', 'хху', 'хіба', 'ц', 'це', 'цебто', 'цей', 'цеп', 'ци', 'цим', 'цими', 'цир', 'цих', 'цло', 'цоб', 'цок', 'цоп', 'цор', 'цс', 'цсс', 'цуг', 'цур', 'цуц', 'цього', 'цьому', 'цю', 'цюк', 'ця', 'цяв', 'цяп', 'ці', 'цід', 'цій', 'цім', 'ціною', 'цією', 'цієї', 'ч', 'чал', 'чар', 'час', 'часто', 'частіше', 'часу', 'чах', 'чей', 'чень', 'через', 'четвертий', 'чи', 'чий', 'чийого', 'чийогось', 'чийому', 'чийомусь', 'чийсь', 'чик', 'чим', 'чимось', 'чимсь', 'чир', 'численна', 'численне', 'численний', 'численним', 'численними', 'численних', 'численні', 'чию', 'чиюсь', 'чия', 'чиясь', 'чиє', 'чиєму', 'чиємусь', 'чиєсь', 'чиєю', 'чиєюсь', 'чиєї', 'чиєїсь', 'чиї', 'чиїй', 'чиїйсь', 'чиїм', 'чиїми', 'чиїмись', 'чиїмось', 'чиїмсь', 'чиїсь', 'чиїх', 'чиїхось', 'чиїхсь', 'чля', 'чого', 'чогось', 'чом', 'чому', 'чомусь', 'чон', 'чоп', 'чортзна', 'чос', 'чотири', 'чотирнадцятий', 'чотирнадцять', 'чу', 'чум', 'чур', 'чш', 'чім', 'чімось', 'чімсь', 'чіт', 'ш', 'ша', 'шаг', 'шал', 'шам', 'шво', 'шед', 'шен', 'шиз', 'шир', 'шляхом', 'шостий', 'шістнадцятий', 'шістнадцять', 'шість', 'щ', 'ще', 'щем', 'щеп', 'щип', 'щир', 'що', 'щоб', 'щоби', 'щодо', 'щойно', 'щоправда', 'щось', 'щі', 'ь', 'ю', 'юз', 'юн', 'юнь', 'юс', 'ют', 'юхт', 'я', 'яв', 'яд', 'яз', 'язь', 'як', 'яка', 'якась', 'якби', 'яке', 'якесь', 'який', 'якийсь', 'яким', 'якими', 'якимись', 'якимось', 'якимсь', 'яких', 'якихось', 'якихсь', 'якого', 'якогось', 'якому', 'якомусь', 'якось', 'якою', 'якоюсь', 'якої', 'якоїсь', 'якраз', 'яку', 'якусь', 'якщо', 'які', 'якій', 'якійсь', 'якім', 'якімсь', 'якісь', 'ял', 'ям', 'ян', 'янь', 'яо', 'яп', 'ярл', 'ясь', 'ять', 'є', 'єр', 'єси', 'і', 'ібн', 'ід', 'із', 'із-за', 'із-під', 'іззаду', 'ізм', 'ізсередини', 'ік', 'ікс', 'ікт', "ім'я", 'імовірно', 'інакша', 'інакше', 'інакший', 'інакшим', 'інакшими', 'інакших', 'інакшого', 'інакшому', 'інакшою', 'інакшої', 'інакшу', 'інакші', 'інакшій', 'інакшім', 'інколи', 'іноді', 'інша', 'інше', 'інший', 'іншим', 'іншими', 'інших', 'іншого', 'іншому', 'іншою', 'іншої', 'іншу', 'інші', 'іншій', 'іншім', 'інь', 'іч', 'іще', 'ї', 'їдь', 'їй', 'їм', 'їх', 'їхнього', 'їхньому', 'їхньою', 'їхньої', 'їхню', 'їхня', 'їхнє', 'їхні', 'їхній', 'їхнім', 'їхніми', 'їхніх', 'її', 'ґ', 'біля', 'читайте', 'якого', 'із', 'який', 'це', 'але', 'зі', 'за', 'також', 'ще', 'на', 'нет', 'яке', 'він', 'його', 'хто', 'що', 'як', 'під', 'яку', 'яких', 'ми', 'яка']
combined_df["text_without_stopwords"] = combined_df["lowercase_text"].apply(
    lambda text: " ".join(
        word for word in text.split() if word not in custom_stop_words
    )
)
combined_df

## Стемінг з допомогою tree_stem

In [ ]:
from tree_stem import stem_word  # Переконайтеся, що модуль tree_stem доступний

def stem_text(row):
    words = row['text_without_stopwords'].split()  # Передбачається, що слова розділені пробілами; можна адаптувати за необхідності
    stemmed_words = [stem_word(word) for word in words]
    return ' '.join(stemmed_words)

# Застосуйте функцію до кожного рядка датафрейму
combined_df['tree_stemmed'] = combined_df.apply(stem_text, axis=1)

combined_df


## Стемінг з допомогою ukr_stemmer

In [ ]:
def stem_text_with_ukrainian_stemmer(row):
    words = row['text_without_stopwords'].split()  # Передбачається, що слова розділені пробілами; можна адаптувати за необхідності
    stemmed_words = [UkrainianStemmer(word).stem_word() for word in words]
    return ' '.join(stemmed_words)

# Застосуйте функцію до кожного рядка датафрейму
combined_df['ukrainian_stemmed'] = combined_df.apply(stem_text_with_ukrainian_stemmer, axis=1)


In [ ]:
combined_df

## Лематизація

In [ ]:
def lemmatize_text(row):
    words = row['text_without_stopwords'].split()  # Передбачається, що слова розділені пробілами; можна адаптувати за необхідності
    lemmatized_words = [lemmatize(word, lang='uk') for word in words]
    return ' '.join(lemmatized_words)

# Застосуйте функцію до кожного рядка датафрейму
combined_df['lemmatized'] = combined_df.apply(lemmatize_text, axis=1)

In [ ]:
combined_df

## Збереження датафрейму в БД

In [ ]:
def update_or_append_dataframe(dataframe, db_file, table_name):
    # З'єднуємося з базою даних SQLite
    conn = sqlite3.connect(db_file)

    # Зчитуємо існуючі дані, якщо таблиця існує
    existing_data_query = f"SELECT * FROM {table_name}"
    existing_data = pd.read_sql_query(existing_data_query, conn)

    # Проходимося по рядках нового DataFrame
    for index, row in dataframe.iterrows():
        # Перевіряємо, чи існує вже рядок з таким же 'link'
        match = existing_data[existing_data['link'] == row['link']]

        if not match.empty:
            # Оновлюємо існуючий рядок, якщо знайдено відповідність
            update_query = f"UPDATE {table_name} SET {', '.join([f'{col} = ?' for col in row.index])} WHERE link = ?"
            conn.execute(update_query, tuple(row.values) + (row['link'],))
        else:
            # Додаємо новий рядок, якщо відповідність не знайдено
            insert_query = f"INSERT INTO {table_name} ({', '.join(row.index)}) VALUES ({', '.join(['?'] * len(row))})"
            conn.execute(insert_query, tuple(row.values))

    # Зберігаємо зміни в базі даних
    conn.commit()
    conn.close()
    print(f"Дані оновлено в таблиці {table_name} у базі даних {db_file}")


In [ ]:
update_or_append_dataframe(combined_df, 'my.db', 'combined_df')

In [2]:
# Замініть 'my.db' на ім'я вашої бази даних
db_path = 'my.db'

# З'єднання з базою даних
conn = sqlite3.connect(db_path)

# Зчитування таблиці з бази даних у датафрейм
test_df = pd.read_sql('SELECT * FROM combined_df', conn)

# Закриття з'єднання
conn.close()

In [3]:
test_df

,time,link,title,views,text,popularity_score,cleaned_text,lowercase_text,text_without_stopwords,tree_stemmed,ukrainian_stemmed,lemmatized
0,23:52,https://www.pravda.com.ua/news/2023/12/5/7431812/,"""Знищують усе"": командувач Повітряних сил опри...",38624,Командувач Повітряних сил генерал-лейтенант ...,86.309524,Командувач Повітряних сил генераллейтенант Мик...,командувач повітряних сил генераллейтенант мик...,командувач повітряних сил генераллейтенант мик...,командувач повітрян сил генераллейтенант микол...,командувач повітрян сил генераллейтенант микол...,командувач повітряний сила генераллейтенант Ми...
1,23:36,https://www.pravda.com.ua/news/2023/12/5/7431795/,"Росіяни запустили ""Шахеди"", повітряна тривога ...",27208,Повітряні сили ЗСУ увечері 5 грудня повідоми...,76.488095,Повітряні сили ЗСУ увечері грудня повідомили п...,повітряні сили зсу увечері грудня повідомили п...,повітряні сили зсу увечері грудня повідомили б...,повітрян зс увечер груд повідом бп україн дже...,повітрян сил зсу увечер грудн повідомил бпла у...,повітряний сила зсу увечері грудень повідомити...
2,23:27,https://www.pravda.com.ua/news/2023/12/5/7431811/,"РосЗМІ: російським військовим наказали ""взяти""...",24437,"Російське видання ""Важные истории"" повідомля...",72.619048,Російське видання Важные истории повідомляє що...,російське видання важные истории повідомляє що...,російське видання важные истории повідомляє ро...,російськ виданн важны истори повідомля родич м...,російськ виданн важны истор повідомля родич мо...,російське видання важные истории повідомляти р...
3,22:57,https://www.pravda.com.ua/news/2023/12/5/7431807/,"Україна працює на всіх рівнях, щоб Конгрес США...",2290,Міністр закордонних справ Дмитро Кулеба заяв...,11.011905,Міністр закордонних справ Дмитро Кулеба заявив...,міністр закордонних справ дмитро кулеба заявив...,міністр закордонних дмитро кулеба заявив украї...,міністр закордонн дмитр кулеба заяв україн пра...,міністр закордонн дмитр кулеб заяв україн прац...,міністр закордонний Дмитро Кулеба заявити укра...
4,22:28,https://www.pravda.com.ua/news/2023/12/5/7431803/,"Посилюємо нашу ППО, зокрема на Одещині – Зелен...",6681,Президент України Володимир Зеленський після...,31.845238,Президент України Володимир Зеленський після з...,президент україни володимир зеленський після з...,президент україни володимир зеленський знищенн...,президент україн володимир зеленськ знищенн ро...,президент україн володимир зеленськ знищенн ро...,президент Україна Володимир Зеленський знищенн...
...,...,...,...,...,...,...,...,...,...,...,...,...
828,01:20,https://censor.net/ua/news/3461464/golova_yevr...,Голова європейської дипломатії Боррель планує ...,709,Голова європейської дипломатії Боррель може ві...,2.000000,Голова європейської дипломатії Боррель може ві...,голова європейської дипломатії боррель може ві...,голова європейської дипломатії боррель відвіда...,голов європейськ дипломат боррел відвіда украї...,гол європейськ дипломаті боррел відвід україн ...,голова європейський дипломатія боррель відвіда...
829,00:52,https://censor.net/ua/news/3461462/bolgarskyyi...,Болгарський парламент почав скасування виняткі...,6041,Болгарські законодавці в середу зробили перший...,21.000000,Болгарські законодавці в середу зробили перший...,болгарські законодавці в середу зробили перший...,болгарські законодавці зробили крок скасування...,болгарськ законодав зроб крок скасуванн винят ...,болгарськ законодавц зробил крок скасуванн вин...,болгарський законодавець зробити крок скасуван...
830,00:24,https://censor.net/ua/news/3461461/bayiden_zap...,"Байден запропонував зміни, які підвищують шанс...",6493,Президент США Джо Байден запропонував змінити ...,23.000000,Президент США Джо Байден запропонував змінити ...,президент сша джо байден запропонував змінити ...,президент сша джо байден запропонував змінити ...,президент сш джо байден запропону змін прикорд...,президент сша джо байден запропонув змінит при...,президент сша джо байден запропонувати змінити...
831,23:5

In [4]:
combined_df = test_df.copy()